In [1]:
import warnings
warnings.filterwarnings('ignore')
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import random as rd
import pickle

In [2]:
networks={}
networks['HINT'] = nx.read_edgelist('1 - output/HINT_Complete.edgelist',delimiter='\t')
networks['IntAct'] = nx.read_edgelist('1 - output/intAct_Significant.edgelist',delimiter='\t')
networks['Reactome'] = nx.read_edgelist('1 - output/Reactome_Significant.edgelist',delimiter='\t')
networks['STRING'] = nx.read_edgelist('1 - output/STRING_Significant.edgelist',delimiter='\t')

## Connected Components, Self Loops and LCC

In [17]:
for name in networks:
    #CCs Info
    G = networks[name]
    ccs = sorted(nx.connected_components(G),key=len,reverse=True)
    print(name)
    print('Top 10:',[len(cc) for cc in ccs][:10])
    print('LCC',len(ccs[0]),'('+str(round(len(ccs[0])/len(G)*100))+'%)')
    #UPDATE G to LCC
    networks[name] = nx.subgraph(networks[name],ccs[0])
    print()

HINT
Top 10: [14763, 4, 3, 3, 2, 2, 2, 2, 2, 2]
LCC 14763 (96%)

IntAct
Top 10: [13268, 5, 3, 3, 3, 3, 3, 3, 3, 2]
LCC 13268 (99%)

Reactome
Top 10: [11873, 9, 8, 8, 6, 6, 5, 5, 5, 5]
LCC 11873 (97%)

STRING
Top 10: [16582, 9, 5, 5, 4, 4, 4, 4, 4, 4]
LCC 16582 (99%)



In [18]:
# Remove Self Loops
for name in networks:
    print(name,networks[name].number_of_edges())
    G = networks[name].copy()
    G.remove_edges_from(
            nx.selfloop_edges(networks[name])
        )
    networks[name]=G
    print(name,networks[name].number_of_edges())

HINT 118901
HINT 114588
IntAct 91578
IntAct 90446
Reactome 228447
Reactome 228447
STRING 252802
STRING 252801


In [5]:
(96+99+97+99)/4

97.75

In [19]:
# Save the LCC
for name in networks:
    G = networks[name]
    nx.write_edgelist(G,"2 - output\\"+name+'.edgelist',delimiter='\t')

## Communities

In [45]:
%%time
comunities={}
for name in networks:
    print(name)
    G = networks[name]
    comunities[name]=sorted([list(com) for com in nx.community.greedy_modularity_communities(G)],key=len,reverse=True)

HINT
HPRD
IntAct
MINT
Reactome
STRING
CPU times: total: 1h 7min 27s
Wall time: 1h 7min 29s


In [46]:
file = open('2 - output/communities.pickle', 'wb')
pickle.dump(comunities, file)
file.close()

In [56]:
del comunities
gc.collect()

63

# Assortativity

In [91]:
def averageNeighborsDegree(G,node):
    neighbors=list(nx.neighbors(G,node))
    if(len(neighbors)==0):
        return 0 
    else:
        return sum([nx.degree(G,n) for n in neighbors])/len(neighbors)

In [94]:
assortativity={}
for name in networks:
    print(name)
    assortativity[name]={}
    G = networks[name]
    nodeWithDegreeK={}
    #Get the nodes with degree 'k'
    for node,k in dict(G.degree).items():
        if k not in nodeWithDegreeK:
            nodeWithDegreeK[k]=[node]
        else:
            nodeWithDegreeK[k].append(node)
    #Get the average neighbors degree for every node with degree'k'
    for k in nodeWithDegreeK:
        means=0
        for node in nodeWithDegreeK[k]:
            means+=averageNeighborsDegree(G,node)
        nodeWithDegreeK[k]=round(means/len(nodeWithDegreeK[k]),2)
    assortativity[name]['distribution']=nodeWithDegreeK
    assortativity[name]['coefficient']=nx.assortativity.degree_assortativity_coefficient(G)

HINT
HPRD
IntAct
MINT
Reactome
STRING


In [95]:
file = open('2 - output/assortativity.pickle', 'wb')
pickle.dump(assortativity, file)
file.close()

# Small World and Diameter

In [116]:
smallWorld={}
for name in networks:
    print(name)
    G = networks[name]
    smallWorld[name]={}
    
    ecc=list(nx.eccentricity(G).values())
    avgSP = round(nx.average_shortest_path_length(G),2)
    diameter=round(max(ecc))

    eccentricityProb={}
    total = len(ecc)
    for c in ecc:
        if c in eccentricityProb:
            eccentricityProb[c]+=1
        else:
            eccentricityProb[c]=1

    for c in eccentricityProb:
        eccentricityProb[c]=eccentricityProb[c]/total

    eccentricityProb = dict(sorted(eccentricityProb.items(), key=lambda x:x[0]))
    
    smallWorld[name]['avgSP']=avgSP
    smallWorld[name]['diameter']=diameter
    smallWorld[name]['ecc']=eccentricityProb

HINT
HPRD
IntAct
MINT
Reactome
STRING


In [117]:
file = open('2 - output/smallWorld.pickle', 'wb')
pickle.dump(smallWorld, file)
file.close()

# Attack and Resilience

In [48]:
#mr is the MatrixOfResults
def getErroSTD(mr):
    #Variance calculation.
    results=[r[1] for r in mr]
    resultsVariance=[]
    #results is a matriz. Each line is a repetion that contains 'howMany' values (howMany is the number of removed nodes)
    #The columns in this matriz are the values found in a point in time (f). So we can calculate the variance for each point 
    numberOfColumns = len(results[0])
    for col in range(numberOfColumns):
        colValues=[]
        for line in results:
            colValues.append(line[col])
        resultsVariance.append(np.std(colValues))   
    return resultsVariance

def rd_impact(G_original,repeat):
    matrizOfResults=[]
    for _ in range(repeat):
        G = G_original.copy()
        Nstart = G.number_of_nodes()
        yAxis=[1]
        xAxis=[0]
        listOfNode = list(G.nodes)
        rd.shuffle(listOfNode)#Randomize the nodes
        for i,_ in enumerate(range(len(listOfNode))):
            G.remove_node(listOfNode[i])
            Nactual = G.number_of_nodes()
            CCs = nx.connected_components(G)
            if Nactual != 0:
                LCC = max(CCs,key=len)
                #𝑃∞(𝑓)/𝑃∞(0)  = len(LCC)/Nstart # Because we start with a connect network, with only one connect component
                impact = len(LCC)/Nstart
            else:
                impact = 0 
            f = 1 - Nactual/Nstart
            yAxis.append(impact)
            xAxis.append(f)
            
        matrizOfResults.append([np.array(xAxis),np.array(yAxis)])
        
    y = np.array(matrizOfResults[0][1])
    for ex in matrizOfResults:
        y+=ex[1]
    y=y/(repeat+1)    
    
    error=getErroSTD(matrizOfResults)
    return matrizOfResults,error,xAxis,y
  
def intentional_impact(G_original,repeat):
    matrizOfResults=[]
    for _ in range(repeat):
        G = G_original.copy()
        Nstart = G.number_of_nodes()
        yAxis=[1]
        xAxis=[0]

        #verticesOrdenados is a list of tuples (verticeName,degree)
        verticesOrdenados = list(dict(sorted(dict(G.degree).items(), key=lambda x:x[1],reverse=True)).items())
        #In case two or more hubs have the same degree
        first=verticesOrdenados[0]
        equalsToFirst=[first[0]]
        cont=1
        while cont<len(verticesOrdenados) and first[1] == verticesOrdenados[cont][1]:
            equalsToFirst.append(verticesOrdenados[cont][0])
            cont+=1
        #randomize the list of hubs that have the same degree
        rd.shuffle(equalsToFirst)
        verticeToRemove=equalsToFirst[0]

        for step in range(Nstart):    
            G.remove_node(verticeToRemove)
            Nactual = G.number_of_nodes()
            CCs = nx.connected_components(G)
            if Nactual != 0:
                LCC = max(CCs,key=len)
                #𝑃∞(𝑓)/𝑃∞(0)  = len(LCC)/Nstart # Because we start with a connect network, with only one connect component
                impact = len(LCC)/Nstart
            else:
                impact = 0 
            f = 1 - Nactual/Nstart
            yAxis.append(impact)
            xAxis.append(f)

            #verticesOrdenados is a list of tuples (verticeName,degree)
            verticesOrdenados = list(dict(sorted(dict(G.degree).items(), key=lambda x:x[1],reverse=True)).items())
            #In case two or more hubs have the same degree
            if(len(verticesOrdenados)>0):
                first=verticesOrdenados[0]
                equalsToFirst=[first[0]]
                cont=1
                while cont<len(verticesOrdenados) and first[1] == verticesOrdenados[cont][1]:
                    equalsToFirst.append(verticesOrdenados[cont][0])
                    cont+=1
                #randomize the list of hubs that have the same degree
                rd.shuffle(equalsToFirst)
                verticeToRemove=equalsToFirst[0]


        matrizOfResults.append([np.array(xAxis),np.array(yAxis)])

    #Calculate the average of all repeat
    y = np.array(matrizOfResults[0][1])
    for ex in matrizOfResults:
        y+=ex[1]
    y=y/(repeat+1)
    
    #Calculate de error and plotIT
    error=getErroSTD(matrizOfResults)
    
    return matrizOfResults,error,xAxis,y

In [56]:
%%time
matrizOfImpacts={}
numberAttacksRandom=30
numberAttacksHub=10

for name in networks:
    print(name)
    G = networks[name]
    LCC = nx.subgraph(G,max(nx.connected_components(G)))
    matrizOfImpacts[name]={}    
    matrizOfImpacts[name]['Random']=rd_impact(LCC,numberAttacksRandom)
    print(name+' Random')
    matrizOfImpacts[name]['Hubs']=intentional_impact(LCC,numberAttacksHub)
    print(name+' Hubs')


HINT
HINT Random
HINT Hubs
HPRD
HPRD Random
HPRD Hubs
IntAct
IntAct Random
IntAct Hubs
MINT
MINT Random
MINT Hubs
Reactome
Reactome Random
Reactome Hubs
STRING
STRING Random
STRING Hubs
CPU times: total: 12h 59min 15s
Wall time: 12h 59min 18s


In [57]:
file = open('2 - output/attackResilience.pickle', 'wb')
pickle.dump(matrizOfImpacts, file)
file.close()